In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from IPython.core.display import display, HTML
#set display options for pandas dataframes
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
display(HTML("<style>.container { width:100% !important; }</style>"))
local_path = "E:/gleif/gleif_parsing/"

c:\program files (x86)\python36-32\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#load GLEIF LEI list
gleif_df = pd.read_csv("../output/lei_info.txt", sep="|", dtype="object")
fill_cols = ["legal_city", "legal_name", "hq_city", "hq_country"]
gleif_df.fillna("-1", inplace=True)
gleif_df = gleif_df[gleif_df.legal_country=="US"] #filter for only US companies to speed matching
#split legal region to capture only 2 digit state code EG: MT
gleif_df['state'] = gleif_df.legal_region.apply(lambda x: x[-2:] if x != "-1" else "")
gleif_df[gleif_df.state!=""].state.head() #check output
#gleif_us = gleif_df[gleif_df.legal_country=="US"]
#gleif_us.to_csv("../output/gleif_usa.txt", index=False, sep="|")


10876    DE
15420    NY
15426    KY
15440    NY
15441    CT
Name: state, dtype: object

In [3]:
#load NIC IDs (this is panel with addresses pulled from NIC)
nic_ids = pd.read_csv(local_path+"data/nic_id_address.txt", sep="|", dtype="object")
nic_ids.fillna("-1", inplace=True)
print(len(nic_ids))
nic_ids.head()

259360


,id_rssd,id_lei,id_tax,nm_lgl,city,state_cd,state_abbr_nm,state_home_cd,street_line1,street_line2,cntry_nm,latest_row_ind,dt_end,dt_start
0,19,0,0,WHITEHALL COLUMBUS BRANCH,WHITEHALL,39,OH,0,4660 EAST MAIN ST.,0,UNITED STATES,Y,99991231,20091231
1,28,0,0,LITTLE RIVER OFFICE,LITTLE RIVER,45,SC,0,1390 HIGHWAY 17 NORTH,0,UNITED STATES,Y,99991231,20101001
2,37,0,0,BANK OF HANCOCK COUNTY,SPARTA,13,GA,0,12855 BROAD STREET,0,UNITED STATES,Y,99991231,20090415
3,46,0,0,BEECHER BRANCH,FLINT,26,MI,0,G-3289 BEECHER ROAD,0,UNITED STATES,Y,99991231,20160816
4,73,0,910591861,UTILITY EMPLOYEES FEDERAL CREDIT UNION,HOQUIAM,53,WA,0,220 MYRTLE STREET,0,UNITED STATES,Y,99991231,20081231


In [4]:
# load id map of 2017 HMDA filers, this is used to get LEI 
id_map = pd.read_csv("../data/id_map.txt", sep="|", dtype="object")
id_map.head()

,agency_code,respondent_id,id_rssd,tax_id_dash,tax_id,arid,institution_id
0,3,11286,271631,59-0238640,590238640,311286,271631
1,3,33542,1914421,87-0489793,870489793,333542,1914421
2,3,1821,554343,35-0318380,350318380,31821,554343
3,5,4122,460079,46-0242124,460242124,54122,460079
4,3,58874,3900950,26-2218160,262218160,358874,3900950


In [5]:
# merge nic ID and address data with panel id map
panel_df = id_map.merge(nic_ids, on="id_rssd", how="left")
panel_df.fillna("-1", inplace=True)
panel_df.id_lei = panel_df.id_lei.apply(lambda x: x if x not in ('0','-1') else '-1')
panel_df.head()

,agency_code,respondent_id,id_rssd,tax_id_dash,tax_id,arid,institution_id,id_lei,id_tax,nm_lgl,city,state_cd,state_abbr_nm,state_home_cd,street_line1,street_line2,cntry_nm,latest_row_ind,dt_end,dt_start
0,3,11286,271631,59-0238640,590238640,311286,271631,-1,590238640,FARMERS & MERCHANTS BANK,MONTICELLO,12,FL,0,200 EAST WASHINGTON ST,0,UNITED STATES,Y,99991231,20120101
1,3,33542,1914421,87-0489793,870489793,333542,1914421,-1,870489793,ROCK CANYON BANK,PROVO,49,UT,0,215 WEST 2230 NORTH,0,UNITED STATES,Y,99991231,20130603
2,3,1821,554343,35-0318380,350318380,31821,554343,-1,350318380,FOWLER STATE BANK,FOWLER,18,IN,0,300 EAST 5TH STREET,0,UNITED STATES,Y,99991231,20120101
3,5,4122,460079,46-0242124,460242124,54122,460079,-1,460242124,HIGHMARK FEDERAL CREDIT UNION,RAPID CITY,46,SD,0,725 FIFTH ST,0,UNITED STATES,Y,99991231,20081231
4,3,58874,3900950,26-2218160,262218160,358874,3900950,-1,262218160,FIRST GREEN BANK,ORLANDO,12,FL,0,250 NORTH ORANGE AVE,0,UNITED STATES,Y,99991231,20171211


In [6]:
# create column with name, address, city, state to match to gleif
panel_df['match_string'] = panel_df.apply(lambda x: x.nm_lgl + " " + x.city + " " + x.state_abbr_nm
                                         + " " + x.street_line1, axis=1)
gleif_df['match_string_gleif'] = gleif_df.apply(lambda x: x.legal_name + " " + x.legal_city + " " +
                                               x.state + " " + x.legal_street1, axis=1)

#nic: nm_lgl, city, state_abbr_nm, street_line1, 
#gleif: legal_name, legal_city, legal_region[-2], legal_street1

In [7]:
panel_df.head()

,agency_code,respondent_id,id_rssd,tax_id_dash,tax_id,arid,institution_id,id_lei,id_tax,nm_lgl,city,state_cd,state_abbr_nm,state_home_cd,street_line1,street_line2,cntry_nm,latest_row_ind,dt_end,dt_start,match_string
0,3,11286,271631,59-0238640,590238640,311286,271631,-1,590238640,FARMERS & MERCHANTS BANK,MONTICELLO,12,FL,0,200 EAST WASHINGTON ST,0,UNITED STATES,Y,99991231,20120101,FARMERS & MERCHANTS BANK MONTICELLO FL 200 EAS...
1,3,33542,1914421,87-0489793,870489793,333542,1914421,-1,870489793,ROCK CANYON BANK,PROVO,49,UT,0,215 WEST 2230 NORTH,0,UNITED STATES,Y,99991231,20130603,ROCK CANYON BANK PROVO UT 215 WEST 2230 NORTH
2,3,1821,554343,35-0318380,350318380,31821,554343,-1,350318380,FOWLER STATE BANK,FOWLER,18,IN,0,300 EAST 5TH STREET,0,UNITED STATES,Y,99991231,20120101,FOWLER STATE BANK FOWLER IN 300 EAST 5TH STREET
3,5,4122,460079,46-0242124,460242124,54122,460079,-1,460242124,HIGHMARK FEDERAL CREDIT UNION,RAPID CITY,46,SD,0,725 FIFTH ST,0,UNITED STATES,Y,99991231,20081231,HIGHMARK FEDERAL CREDIT UNION RAPID CITY SD 72...
4,3,58874,3900950,26-2218160,262218160,358874,3900950,-1,262218160,FIRST GREEN BANK,ORLANDO,12,FL,0,250 NORTH ORANGE AVE,0,UNITED STATES,Y,99991231,20171211,FIRST GREEN BANK ORLANDO FL 250 NORTH ORANGE AVE


In [8]:
#Save combined data before beginning matching
panel_df[panel_df.id_lei!="-1"] #remove blank LEIs
panel_df.to_csv("../output/panel_lei_map.txt", index=False, sep="|")


In [9]:
#match panel to gleif for rows with an LEI present (and a match string)
#use this to test output
#Conditions: 
# LEI != -1
#match string != -4
#test results indicate that a match score of 95 or better is usable for matching
#several mistakes in the NIC LEI data were discovered
#these mistakes usually involved character transposition EG 0 to O

#panel_df['match_tup'] = panel_df[(panel_df.id_lei!="-1")&(panel_df.match_string!="-4")].apply(
#lambda x: process.extractOne(x.match_string, gleif_df.match_string_gleif),axis=1)
panel_df.head()

,agency_code,respondent_id,id_rssd,tax_id_dash,tax_id,arid,institution_id,id_lei,id_tax,nm_lgl,city,state_cd,state_abbr_nm,state_home_cd,street_line1,street_line2,cntry_nm,latest_row_ind,dt_end,dt_start,match_string
0,3,11286,271631,59-0238640,590238640,311286,271631,-1,590238640,FARMERS & MERCHANTS BANK,MONTICELLO,12,FL,0,200 EAST WASHINGTON ST,0,UNITED STATES,Y,99991231,20120101,FARMERS & MERCHANTS BANK MONTICELLO FL 200 EAS...
1,3,33542,1914421,87-0489793,870489793,333542,1914421,-1,870489793,ROCK CANYON BANK,PROVO,49,UT,0,215 WEST 2230 NORTH,0,UNITED STATES,Y,99991231,20130603,ROCK CANYON BANK PROVO UT 215 WEST 2230 NORTH
2,3,1821,554343,35-0318380,350318380,31821,554343,-1,350318380,FOWLER STATE BANK,FOWLER,18,IN,0,300 EAST 5TH STREET,0,UNITED STATES,Y,99991231,20120101,FOWLER STATE BANK FOWLER IN 300 EAST 5TH STREET
3,5,4122,460079,46-0242124,460242124,54122,460079,-1,460242124,HIGHMARK FEDERAL CREDIT UNION,RAPID CITY,46,SD,0,725 FIFTH ST,0,UNITED STATES,Y,99991231,20081231,HIGHMARK FEDERAL CREDIT UNION RAPID CITY SD 72...
4,3,58874,3900950,26-2218160,262218160,358874,3900950,-1,262218160,FIRST GREEN BANK,ORLANDO,12,FL,0,250 NORTH ORANGE AVE,0,UNITED STATES,Y,99991231,20171211,FIRST GREEN BANK ORLANDO FL 250 NORTH ORANGE AVE


In [ ]:
count = 0
panel_df["match_tup"] = ""
for index, row in panel_df[panel_df.match_string!="-1 -1 -1 -1"].iterrows():
    print(index, row.match_string)
    #match_output = row.apply(lambda x: process.extractOne(x.match_string, gleif_df.match_string_gleif))
    match_output = process.extractOne(panel_df.at[index, "match_string"], gleif_df.match_string_gleif)
    #print(panel_df.at[index, "match_string"])
    panel_df.at[index, "match_tup"] = match_output
    
    if count % 10 == 0 and count >=10:
        panel_df.to_csv("../output/panel_results_"+str(count)+".txt", sep="|", index=False)
        print("saving {count}".format(count=count))


0 FARMERS & MERCHANTS BANK MONTICELLO FL 200 EAST WASHINGTON ST


#old match logic, estimated 20 days to complete
#changing to loop to enable file writing every n rows
panel_df['match_all_tup'] = panel_df[(panel_df.match_string!="-4")&(panel_df.lei=="-1")].apply(
    lambda x: process.extractOne(x.match_string, gleif_df.match_string_gleif),axis=1)


panel_df.to_csv("../output/full_test.txt", index=False, sep="|")

In [ ]:
nic_ids.head()

In [ ]:
#split match tuple
print(panel_df.at[1, "match_"])

In [ ]:
#add flag for in NIC